# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Step2 加载数据

In [2]:
import pandas as pd

data = pd.read_csv("/home/xiaoyicheng/Code_Hub/HuggingFace/Basic/3-model/data/ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


## Step3 创建Dataset

In [4]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("/home/xiaoyicheng/Code_Hub/HuggingFace/Basic/3-model/data/ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [5]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [6]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [7]:
for i in range(10):
    print(trainset[i])

('虽然房价贵了点，但是感觉是太原最好的宾馆了。整体感觉很好，就是周围没有什么好的餐馆。', 1)
('在甘肃能有这样一个酒店实属不易，比阳光大厦好很多，无论从硬件设施还是服务水平都一流，建议：游泳池太冷，房费含上早餐', 1)
('非常糟糕的服务态度,隔音非常差即使是在高层夜晚有火车声第二天搬到远洲国际', 0)
('离杭州不少路,酒店特别是晚上不好找,游乐项目不多宾馆反馈2007年3月5日：酒店已增加了园区内电瓶车观光游览、垂钓、各种球类运动项目、射箭、野外拓展、农家土柴灶自己烧、烧烤等等活动。', 1)
('300多的价格入住这家佛山历史最悠久的酒店也真是值得，起码起到缅怀历史的作用，要知道它是广东第一家具有楼顶国产旋转餐厅的酒店。推荐这家酒店吃的地道，很具佛山本地特色，不管是二楼中餐还是楼顶的旋转餐厅用早茶、午饭、晚饭都可以，价格平民化，不像一些带个旋转就一定来个套餐。如果是初次来佛山，推荐入住这里。酒店服务设施难免老化，但是还过得去吧。房间10多平米，不大；附近就是佳宁娜听说那里的房间更加小；附件游览的地方是百花广场（商业广场），祖庙', 1)
('非常好的酒店，非常好的服务，我先生对酒店一向非常挑剔，只有这次他实在没话说！补充点评2008年8月6日：酒店餐厅正在装修，但不影响住宿。', 1)
('性价比还不错，对面就是新光天地百货，逛街很不错！', 1)
('入住了2间行政湖景房2个晚上,感觉房间有点小,但是设计还不错.卫生间和四星的行政房来比就真是小了不少.电视也是很旧款很小的那种.景色是真的不错.出去散步也很方便.服务就要看人了,CHECK-IN的时候明明说了要求要吸烟楼层,前台居然骗我说行政楼层都是非吸烟的.进了电梯才发现只有我住的24层是行政楼层的第一层,也是非吸烟楼层.25-28楼都是可以吸烟的.在这件事情上弄的很不愉快.宾馆反馈2008年3月5日：非常感谢您选择入住金雁酒店，并对酒店服务提出了宝贵意见。在此表示最诚挚的歉意。经过调查了解，主要原因是该员工接待经验不足,为求快速为您办理入住,在您提出是否行政楼层都是非吸烟的疑问时未能严谨回答。我们将对该员工进行语言表述的培训.再次谢谢您的宝贵意见，衷心期待您的再次光临！', 1)
('比较清静，环境还好，我住的一楼房间比较潮湿有霉味，蚊子较多', 1)
('最好别去,很差,看完很差想换酒店,他们

## Step5 创建Dataloader

In [8]:
import os
os.environ['http_proxy'] = "http://127.0.0.1:7890"
os.environ['https_proxy'] = "http://127.0.0.1:7890"


In [9]:
import torch

tokenizer = AutoTokenizer.from_pretrained("rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [10]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [11]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[ 101, 1963, 3362,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        ...,
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 1184, 1378,  ...,    0,    0,    0],
        [ 101, 6821, 3221,  ..., 3121,  857,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 1

## Step6 创建模型及优化器

In [12]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("rbt3")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [14]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

## Step8 模型训练

In [15]:
train()

ep: 0, global_step: 0, loss: 0.7068737149238586
ep: 0, global_step: 100, loss: 0.26421037316322327
ep: 0, global_step: 200, loss: 0.3634519875049591
ep: 0, acc: 0.8788659572601318
ep: 1, global_step: 300, loss: 0.24061305820941925
ep: 1, global_step: 400, loss: 0.338666170835495
ep: 1, acc: 0.89304119348526
ep: 2, global_step: 500, loss: 0.3848312795162201
ep: 2, global_step: 600, loss: 0.1734195500612259
ep: 2, acc: 0.8917525410652161


## Step9 模型预测

In [20]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(logits)
    print(pred.item())
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

tensor([[-2.6905,  2.6931]], device='cuda:0')
1
输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [21]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [22]:
pipe(sen)

[{'label': '好评！', 'score': 0.9954296350479126}]